<a href="https://colab.research.google.com/github/jaab/AI---Deep-Learning-com-PyTorch-e-Python/blob/main/Ptroject_3_brest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PROJECTO 3 CLASSIFICAÇÃO BINÁRIA BREST CANCER COM TUNING DOS PARAMETROS
 Etapa 1: Importação das bibliotecas

In [ ]:
!pip install skorch

In [ ]:
import pandas as pd
import numpy as np
import torch.nn as nn
from skorch import NeuralNetBinaryClassifier
import torch
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV
torch.__version__

'2.1.0+cu121'

Etapa 2 : Base de Dados

In [ ]:
np.random.seed(123)
torch.manual_seed(123)

In [ ]:
previsores = pd.read_csv('entradas_breast.csv')
classe = pd.read_csv('saidas_breast.csv')

In [ ]:
previsores = np.array(previsores, dtype='float32')
classe = np.array(classe, dtype='float32').squeeze(1)

In [ ]:
previsores.shape

(569, 30)

In [ ]:
classe.shape

(569,)

In [ ]:
previsores,classe

(array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e+02, 2.750e+02,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e+02, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e+02, 4.087e-01,
         1.240e+02],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1.,
        1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1.,
        1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1.,
        0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0.

Etapa 3: Classe para estrutura da rede neural

In [ ]:
#passando parametros para a função
class classificador_torch(nn.Module):
  def __init__(self,activation,neurons,initializer):
    super().__init__()

    # 30 -> 16 -> 16 -> 1
    self.dense0 = nn.Linear(30,neurons)
    initializer(self.dense0.weight)
    self.activation0 = activation

    self.dense1 = nn.Linear(neurons,neurons)
    initializer(self.dense1.weight)
    self.activation1 = activation

    self.dense2 = nn.Linear(neurons,1)
    initializer(self.dense2.weight)
    self.activation2 = activation

    self.output = nn.Sigmoid()

  def forward(self,x):
    x = self.dense0(x)
    x = self.activation0(x)
    x = self.dense1(x)
    x = self.activation1(x)
    x = self.dense2(x)
    x = self.activation2(x)
    x = self.output(x)
    return x

Etapa 4: Skorch

In [ ]:
classificador_sklearn = NeuralNetBinaryClassifier(
    module=classificador_torch,
    lr=0.001,
    optimizer__weight_decay=0.0001,
    train_split=False
    )

Etapa 5: Tuning dos parametros

In [26]:
params = {
    'batch_size': [10],
    'max_epochs': [100],
    'optimizer': [torch.optim.Adam, torch.optim.SGD],
    'criterion': [torch.nn.BCELoss, torch.nn.HingeEmbeddingLoss],
    'module__activation': [F.relu,F.tanh],
    'module__neurons': [8,16],
    'module__initializer': [torch.nn.init.uniform_, torch.nn.init.normal_]
    }

In [27]:
params

{'batch_size': [10],
 'max_epochs': [100],
 'optimizer': [torch.optim.adam.Adam, torch.optim.sgd.SGD],
 'criterion': [torch.nn.modules.loss.BCELoss,
  torch.nn.modules.loss.HingeEmbeddingLoss],
 'module__activation': [<function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
  <function torch.nn.functional.tanh(input)>],
 'module__neurons': [8, 16],
 'module__initializer': [<function torch.nn.init.uniform_(tensor: torch.Tensor, a: float = 0.0, b: float = 1.0) -> torch.Tensor>,
  <function torch.nn.init.normal_(tensor: torch.Tensor, mean: float = 0.0, std: float = 1.0) -> torch.Tensor>]}

In [28]:
grid_search =GridSearchCV(estimator=classificador_sklearn, param_grid=params, scoring='accuracy', cv=5)
grid_search = grid_search.fit(previsores, classe)

Streaming output truncated to the last 5000 lines.
      1       36.9581  0.0743
      2       36.7399  0.0799
      3       36.9551  0.0817
      4       36.9535  0.0815
      5       36.9566  0.0800
      6       36.9581  0.0803
      7       37.1809  0.0809
      8       37.1809  0.0818
      9       37.1809  0.0751
     10       37.1809  0.0901
     11       37.1825  0.0789
     12       36.9657  0.0778
     13       36.9657  0.0812
     14       37.1660  0.0822
     15       37.3794  0.0845
     16       37.3794  0.0857
     17       37.3794  0.0833
     18       37.3703  0.0835
     19       37.3703  0.0792
     20       37.3703  0.0820
     21       37.3703  0.0847
     22       37.3703  0.0882
     23       37.3703  0.0830
     24       37.3703  0.0838
     25       37.3703  0.0781
     26       37.3703  0.0807
     27       37.3703  0.1156
     28       37.3703  0.1414
     29       37.3703  0.1140
     30       37.3703  0.1125
     31       37.3703  0.1096
     32       37.37

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/skorch/classifier.py", line 381, in predict
    return (y_proba[:, 1] > self.threshold).astype('uint8')
IndexError: too many indi

A saída de streaming foi truncada nas últimas 5000 linhas.
-------  ------------  ------
      1        0.7648  0.0842
      2        0.8044  0.0872
      3        0.8505  0.0903
      4        0.8890  0.0911
      5        0.8538  0.0894
      6        0.7505  0.0925
      7        0.7198  0.0925
      8        0.7209  0.0922
      9        0.7209  0.0908
     10        0.7220  0.0979
     11        0.7220  0.0864
     12        0.7253  0.0882
     13        0.7253  0.0906
     14        0.7275  0.0870
     15        0.7275  0.0874
     16        0.7273  0.0931
     17        0.7176  0.0846
     18        0.7187  0.0966
     19        0.7187  0.0957
     20        0.7198  0.0823
     21        0.7198  0.1023
     22        0.7163  0.1001
     23        0.7143  0.0870
     24        0.7143  0.0879
     25        0.7143  0.0903
     26        0.7143  0.0924
     27        0.7143  0.0970
     28        0.7143  0.0895
     29        0.7165  0.0949
     30        0.7187  0.0938
     31    

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [0.53631424 0.6274181  0.47303214 0.6274181  0.45853128 0.50787145
 0.51839776 0.55548828 0.6274181  0.6274181  0.61689179 0.6274181
 0.66783108 0.56244372 0.73982301 0.53445117        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]
  warnings.warn(


      2        0.6834  0.1066
      3        0.6681  0.1113
      4        0.6637  0.1010
      5        0.6626  0.0983
      6        0.6652  0.0978
      7        0.6579  0.0947
      8        0.6530  0.1021
      9        0.6510  0.0979
     10        0.6517  0.1001
     11        0.6515  0.0969
     12        0.6455  0.1000
     13        0.6419  0.1131
     14        0.6389  0.1013
     15        0.6384  0.1015
     16        0.6390  0.0933
     17        0.6301  0.1056
     18        0.6271  0.0985
     19        0.6214  0.1026
     20        0.6212  0.0980
     21        0.6177  0.1011
     22        0.6178  0.0993
     23        0.6181  0.1159
     24        0.6162  0.1061
     25        0.6156  0.1003
     26        0.6151  0.0960
     27        0.6111  0.1029
     28        0.6354  0.0995
     29        0.6274  0.1041
     30        0.6180  0.0978
     31        0.5791  0.0981
     32        0.6230  0.1026
     33        0.6097  0.1274
     34        0.6121  0.1136
     35   

In [23]:
melhores_parametros = grid_search.best_params_
melhor_percisao = grid_search.best_score_

In [29]:
melhores_parametros

{'batch_size': 10,
 'criterion': torch.nn.modules.loss.BCELoss,
 'max_epochs': 50,
 'module__activation': <function torch.nn.functional.tanh(input)>,
 'module__initializer': <function torch.nn.init.normal_(tensor: torch.Tensor, mean: float = 0.0, std: float = 1.0) -> torch.Tensor>,
 'module__neurons': 16,
 'optimizer': torch.optim.adam.Adam}

In [30]:
melhor_percisao

0.6889300716580182